In [4]:
%load_ext autoreload
%autoreload 2
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import warnings
import time
warnings.filterwarnings("ignore")
import sys
sys.path.append('../source/')
sys.path.append('../methods_configuration/')
from precomputing import read_countrydata, read_countries, repair_increasing     
from scipy import stats 
import matplotlib.colors as colors   
 
import os  
import getopt, sys    
import datetime
from datetime import date, timedelta  
from smoothing import piecewise_STL, simple_mirroring 
from forecast_one_country import forecast_one_country
from function_configuration import METHODS   
from paper_compute_forecasts import parse_col_links, get_methods_params, check_existance, create_folders, extrapolate_linearly_weekly

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
dates = ["2022-01-11","2022-01-11"] 
countries = ["Egypt", "Egypt"] 
 
methods,kwargs_list,names,cv_flag = get_methods_params([3],True)  
retro_data_info = pd.read_csv("../data/JHU_data_links.csv")   
retro_data_info.columns = ["date"] + list(retro_data_info.columns)[1:] 
#-----------------full scenario---------------------------------
horisons = [7,14] 
data_cols = ["global-confirmed","global-deaths"]  
smoothing_fun = piecewise_STL  
subset=[3] 

ax=[]


fig = plt.figure(figsize=(17, 3), dpi=1200)
ax.append(plt.subplot2grid((2, 1), (0, 0)))
ax.append(plt.subplot2grid((2, 1), (1, 0))) 


i=0
H=7
title_ = ["(a)","(b)","(c)","(d)","(e)"]

fs = 12
cc  = "#808080"
with plt.rc_context({'axes.edgecolor':cc, 'xtick.color':cc, 'ytick.color':cc, 'figure.facecolor':'white'}):

    for data_col, country, date in zip(data_cols, countries, dates): 
        datasource, parse_column, typedata = parse_col_links(data_col)   
        link_for_forecast  = retro_data_info[retro_data_info["date"]==str(date)[:10]][data_col].values[0]
        df_for_forecast = pd.read_csv(link_for_forecast)  
        cumulative_list, date_list = read_countries(df_for_forecast, [country], 0, datasource, typedata=typedata)   
        cumulative_cases, dates_ = cumulative_list[0], date_list[0]
        cumulative_cases_ = repair_increasing(cumulative_cases)  
        return_dict, _, ci_full = forecast_one_country({}, country, cumulative_cases, dates_,
                                                       methods, kwargs_list, names, smoothing_fun, 
                                                       datasource, H=H, return_val=True, saveplot=False, newly=True) 
        smoothed = np.ravel(return_dict[country][-2]["daily_smoothed"].values[::-1].copy()) 
        dates_plot = np.ravel(return_dict[country][-2]["date"].values[::-1].copy())  
        
        
        ax[i].plot(range(len(smoothed)), smoothed,c="red")
        #---------point forecast---------------------------------
 
        ax[i].bar(range(len(cumulative_cases_)-1),np.diff(cumulative_cases_),color="#66C3A6",alpha=0.95) 

 
        interv=14
        start_plot=140
        labels = ['Aug 29', 'Sep 12', 'Sep 26', 'Oct 10', 'Oct 24', 'Nov 7', 'Nov 21', "Dec 5","Dec 19","Jan 2"][::-1] 
        x = range(len(cumulative_cases_)-10,len(cumulative_cases_)-start_plot-11,-interv)
        
        ax[i].set_xlim([len(cumulative_cases_)-start_plot,len(cumulative_cases_)-1])

        #ax[i].set_ylim([0,1.1*max(cumulative_cases[-start_plot-H:])])

        ax[i].set_xticks(x)
        ax[i].set_xticklabels(np.array(labels), fontsize=fs)
  


        ax[i].set_frame_on(False)
        ax[i].axes.get_xaxis().set_visible(True)
        ax[i].grid(axis='y',alpha=0.5)
        for label in ax[i].get_xticklabels():
            #label.set_rotation(60)
            label.set_horizontalalignment('right')
        ax[i].set_ylabel(title_[i], fontsize=fs, rotation=0,labelpad=10) 
        i+=1
    fig.savefig('egypt.pdf') 

In [12]:
dates = ["2020-09-10","2021-03-12","2021-03-12","2021-03-12","2021-03-12"] 
countries = ["Spain", "Brazil", "Kansas","China","Germany"]


 
methods,kwargs_list,names,cv_flag = get_methods_params([3],True)  
retro_data_info = pd.read_csv("../data/JHU_data_links.csv")   
retro_data_info.columns = ["date"] + list(retro_data_info.columns)[1:] 
#-----------------full scenario---------------------------------
horisons = [7,14] 
data_cols = ["global-confirmed","global-confirmed", "US-confirmed","global-confirmed","global-confirmed" ] 
smoothing_fun = piecewise_STL  
subset=[3] 

ax=[]
fig = plt.figure(figsize=(24, 4), dpi=1200)
ax.append(plt.subplot2grid((3, 3), (0, 0), colspan=2))
ax.append(plt.subplot2grid((3, 3), (1, 0)))
ax.append(plt.subplot2grid((3, 3), (1, 1)))
ax.append(plt.subplot2grid((3, 3), (2, 0)))
ax.append(plt.subplot2grid((3, 3), (2, 1)))
i=0
H=7
title_ = ["(a)","(b)","(c)","(d)","(e)"]

fs = 12
cc  = "#808080"
with plt.rc_context({'axes.edgecolor':cc, 'xtick.color':cc, 'ytick.color':cc, 'figure.facecolor':'white'}):

    for data_col, country, date in zip(data_cols, countries, dates): 
        datasource, parse_column, typedata = parse_col_links(data_col)   
        link_for_forecast  = retro_data_info[retro_data_info["date"]==str(date)[:10]][data_col].values[0]
        df_for_forecast = pd.read_csv(link_for_forecast)  
        cumulative_list, date_list = read_countries(df_for_forecast, [country], 0, datasource, typedata=typedata)   
        cumulative_cases, dates_ = cumulative_list[0], date_list[0]
        cumulative_cases_ = repair_increasing(cumulative_cases)  
        return_dict, _, ci_full = forecast_one_country({}, country, cumulative_cases, dates_,
                                                       methods, kwargs_list, names, smoothing_fun, 
                                                       datasource, H=H, return_val=True, saveplot=False, newly=True) 

        #---------point forecast---------------------------------

        smoothed = np.ravel(return_dict[country][-2]["daily_smoothed"].values[::-1].copy()) 
        dates_plot = np.ravel(return_dict[country][-2]["date"].values[::-1].copy())  
        ax[i].bar(range(len(cumulative_cases_)-1),np.diff(cumulative_cases_),color="#66C3A6",alpha=0.95)
        ax[i].plot(range(len(smoothed)), smoothed,c="red")
        ax[i].bar(range(len(smoothed)-7,len(smoothed)), smoothed[-7:],color="#8EA2CC",alpha=0.95)

        if i==0:
            interv=30
            start_plot=235
            labels = ['Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020'][::-1] 
            x = range(len(dates_plot)-10-H,len(dates_plot)-start_plot-10-H,-interv)
        else:
            interv=14
            start_plot=90
            labels = ['Dec 20', 'Jan 3', 'Jan 17', 'Jan 31', 'Feb 14', 'Feb 28', 'Mar 14'][::-1] 
            x = range(len(dates_plot)-2,len(dates_plot)-start_plot-2,-interv)
        ax[i].set_xlim([len(smoothed)-start_plot,len(smoothed)])

        #ax[i].set_ylim([0,1.1*max(cumulative_cases[-start_plot-H:])])

        ax[i].set_xticks(x)
        ax[i].set_xticklabels(np.array(labels), fontsize=fs)

        if country=="Spain":
            y = [0,10000,20000]
            ylabels = ["0", "10k", "20k"]
            ax[i].set_yticks(y)
            ax[i].set_yticklabels(np.array(ylabels), fontsize=fs )   
        elif country=="Brazil":
            y = [0,20000,40000,60000,80000]
            ylabels = ["0", "20k", "40k", "60k", "80k"] 
            ax[i].set_yticks(y)
            ax[i].set_yticklabels(np.array(ylabels), fontsize=fs)     
        elif country=="Germany":
            y = [0,10000,20000,30000,40000]
            ylabels = ["0", "10k", "20k", "30k", "40k"]
            ax[i].set_yticks(y)
            ax[i].set_yticklabels(np.array(ylabels), fontsize=fs )    
        else: 
            ax[i].set_ylim([0,1.1*max(np.diff(cumulative_cases_)[-start_plot-H:])])


        ax[i].set_frame_on(False)
        ax[i].axes.get_xaxis().set_visible(True)
        ax[i].grid(axis='y',alpha=0.5)
        for label in ax[i].get_xticklabels():
            #label.set_rotation(60)
            label.set_horizontalalignment('right')
        ax[i].set_ylabel(title_[i], fontsize=fs, rotation=0,labelpad=10) 
        i+=1
    fig.savefig('trends.pdf') 